### CELDA 1: Importar librerías 

# 1.0 - Análisis Exploratorio de Datos (EDA) de Crímenes en NYC

**Objetivo:** El objetivo de este notebook es realizar un análisis exploratorio (EDA) de los datos crudos de denuncias de crímenes de la Ciudad de Nueva York.

Usaremos este análisis para:
1.  Entender la distribución y los patrones de los crímenes.
2.  Identificar las características (`features`) más prometedoras para nuestro modelo.
3.  Validar la variable objetivo (`target`) que definiremos para la predicción.

In [ ]:
# CELDA 1: Configuración del Entorno 

import matplotlib
# Le decimos a Matplotlib que use un backend no interactivo compatible con notebooks
matplotlib.use('Agg') 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import folium
from folium.plugins import HeatMap

# Configuramos el estilo de los gráficos para que se vean mejor
sns.set_theme(style="whitegrid")
# Configuramos pandas para que nos muestre más columnas
pd.set_option('display.max_columns', 50)

print("Librerías importadas y backend de Matplotlib configurado a 'Agg'.")

Librerías importadas y backend de Matplotlib configurado a 'Agg'.


## 2. Carga y Limpieza Inicial de Datos

Cargamos el dataset crudo desde `data/raw/`. Realizamos una limpieza mínima indispensable (manejo de fechas y nulos) para poder realizar el análisis. Este paso es solo para exploración; la limpieza final y el preprocesamiento se automatizarán en el script `src/preprocessing.py`.

In [13]:
# --- INICIO DEL PROCESO DE CARGA Y LIMPIEZA ---
import pandas as pd
import numpy as np
import os

# --- CORRECCIÓN DE LÓGICA ---
# El notebook de EDA debe usar los datos CRUDOS (raw) para explorar TODAS las columnas.
# El pipeline (src/*) usa estos descubrimientos para crear los datos PROCESADOS.

raw_file = '../data/raw/complaints.csv'

try:
    print(f"Paso 1: Cargando datos crudos de '{raw_file}' para exploración...")
    df_eda = pd.read_csv(raw_file, low_memory=False)
    print(" -> ¡Éxito! Datos cargados.")

    print("\nPaso 2: Estandarizando nombres de columnas a minúsculas...")
    df_eda.columns = df_eda.columns.str.lower()
    print(" -> ¡Éxito! Nombres estandarizados.")

    print("\nPaso 3: Realizando limpieza básica (solo para EDA)...")
    # Limpieza mínima necesaria para que el notebook funcione
    df_eda['cmplnt_fr_dt'] = pd.to_datetime(df_eda['cmplnt_fr_dt'], errors='coerce')
    df_eda.dropna(subset=['cmplnt_fr_dt', 'latitude', 'longitude', 'ofns_desc', 'prem_typ_desc'], inplace=True)
    print(" -> Filas con valores nulos clave eliminadas.")

    # --- DIAGNÓSTICO DE PRECISIÓN ---
    # Creamos la variable objetivo aquí para analizarla
    print("\nPaso 4: Creando variable objetivo 'is_robbery' para análisis...")
    df_eda['is_robbery'] = np.where(df_eda['ofns_desc'].str.contains('ROBBERY', na=False), 1, 0)
    print(" -> ¡Éxito! Variable 'is_robbery' creada.")

except FileNotFoundError:
    print(f"\n!!! ERROR CRÍTICO: No se encontró el archivo '{raw_file}'.")
    print("    Asegúrate de haber ejecutado el pipeline 'python -m src.pipeline' primero para descargar los datos.")
    df_eda = pd.DataFrame() # DataFrame vacío para evitar errores

# --- ANÁLISIS DE PRECISIÓN (Desbalance de Clases) ---
if not df_eda.empty:
    print("\n--- Análisis de Desbalance de Clases (Precisión) ---")
    target_counts = df_eda['is_robbery'].value_counts()
    target_percent = df_eda['is_robbery'].value_counts(normalize=True) * 100
    
    print("Conteo de la variable objetivo 'is_robbery':")
    print(target_counts)
    print("\nPorcentaje de la variable objetivo 'is_robbery':")
    print(target_percent)
    
    # Mostramos un gráfico de barras
    plt.figure(figsize=(8, 5))
    # --- CORRECCIÓN ESTÉTICA (Warning) ---
    sns.barplot(x=target_percent.index, y=target_percent.values, palette='viridis', hue=target_percent.index, legend=False)
    plt.title('Distribución de la Variable Objetivo (is_robbery)', fontsize=16)
    plt.ylabel('Porcentaje (%)')
    plt.xlabel('Clase (0 = No Robo, 1 = Robo)')
    plt.xticks([0, 1])
    plt.show()
else:
    print("\nNo se pudieron cargar datos para el análisis.")

df_eda.head()

Paso 1: Cargando datos crudos de '../data/raw/complaints.csv' para exploración...
 -> ¡Éxito! Datos cargados.

Paso 2: Estandarizando nombres de columnas a minúsculas...
 -> ¡Éxito! Nombres estandarizados.

Paso 3: Realizando limpieza básica (solo para EDA)...
 -> Filas con valores nulos clave eliminadas.

Paso 4: Creando variable objetivo 'is_robbery' para análisis...
 -> ¡Éxito! Variable 'is_robbery' creada.

--- Análisis de Desbalance de Clases (Precisión) ---
Conteo de la variable objetivo 'is_robbery':
is_robbery
0    48685
1     1306
Name: count, dtype: int64

Porcentaje de la variable objetivo 'is_robbery':
is_robbery
0    97.38753
1     2.61247
Name: proportion, dtype: float64


C:\Users\Diegosuar\AppData\Local\Temp\ipykernel_22136\62528991.py:57: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


,cmplnt_num,cmplnt_fr_dt,cmplnt_fr_tm,cmplnt_to_dt,cmplnt_to_tm,addr_pct_cd,rpt_dt,ky_cd,ofns_desc,pd_cd,pd_desc,crm_atpt_cptd_cd,law_cat_cd,boro_nm,loc_of_occur_desc,prem_typ_desc,juris_desc,jurisdiction_code,parks_nm,hadevelopt,housing_psa,x_coord_cd,y_coord_cd,susp_age_group,susp_race,susp_sex,transit_district,latitude,longitude,lat_lon,patrol_boro,station_name,vic_age_group,vic_race,vic_sex,is_robbery
0,298704289,2024-12-31,22:00:00,2024-12-31T00:00:00.000,22:10:00,101,2024-12-31T00:00:00.000,106,FELONY ASSAULT,109.0,"ASSAULT 2,1,UNCLASSIFIED",COMPLETED,FELONY,QUEENS,INSIDE,RESIDENCE - APT. HOUSE,N.Y. POLICE DEPT,0,(null),(null),NaN,1055868.0,156521.0,UNKNOWN,UNKNOWN,F,NaN,40.596011,-73.742116,"(40.596011, -73.742116)",PATROL BORO QUEENS SOUTH,(null),25-44,UNKNOWN,M,0
1,298697982,2024-12-31,13:46:00,2024-12-31T00:00:00.000,13:46:00,103,2024-12-31T00:00:00.000,578,HARRASSMENT 2,638.0,"HARASSMENT,SUBD 3,4,5",COMPLETED,VIOLATION,QUEENS,(null),STREET,N.Y. POLICE DEPT,0,(null),(null),NaN,1041424.0,195842.0,UNKNOWN,BLACK,M,NaN,40.704033,-73.793792,"(40.704033064999024, -73.79379204757792)",PATROL BORO QUEENS SOUTH,(null),25-44,BLACK,F,0
2,298698018,2024-12-31,11:30:00,2024-12-31T00:00:00.000,11:35:00,120,2024-12-31T00:00:00.000,344,ASSAULT 3 & RELATED OFFENSES,101.0,ASSAULT 3,COMPLETED,MISDEMEANOR,STATEN ISLAND,FRONT OF,STREET,N.Y. POLICE DEPT,0,(null),(null),NaN,960216.0,173272.0,UNKNOWN,UNKNOWN,M,NaN,40.642245,-74.086602,"(40.642245, -74.086602)",PATROL BORO STATEN ISLAND,(null),25-44,BLACK,M,0
3,298699443,2024-12-31,15:20:00,2024-12-31T00:00:00.000,15:30:00,48,2024-12-31T00:00:00.000,106,FELONY ASSAULT,109.0,"ASSAULT 2,1,UNCLASSIFIED",COMPLETED,FELONY,BRONX,INSIDE,RESIDENCE - APT. HOUSE,N.Y. POLICE DEPT,0,(null),(null),NaN,1013693.0,250702.0,45-64,BLACK,M,NaN,40.854755,-73.893568,"(40.854755, -73.893568)",PATROL BORO BRONX,(null),25-44,BLACK,F,0
4,298698001,2024-12-30,22:00:00,2024-12-31T00:00:00.000,01:00:00,109,2024-12-31T00:00:00.000,342,PETIT LARCENY OF MOTOR VEHICLE,350.0,"LARCENY, PETIT OF MOPED",COMPLETED,MISDEMEANOR,QUEENS,INSIDE,PARKING LOT/GARAGE (PUBLIC),N.Y. POLICE DEPT,0,(null),(null),NaN,1031482.0,213641.0,(null),(null),(null),NaN,40.752954,-73.829522,"(40.752954, -73.829522)",PATROL BORO QUEENS NORTH,(null),18-24,WHITE HISPANIC,M,0


### Hallazgo Clave: Selección del Problema de ML

El gráfico de barras anterior (`is_robbery`) reveló un **desbalance de clases extremo (97% vs 3%)**. Intentar predecir un evento tan raro con features tan generales (como la hora y el lugar) demostró ser ineficaz (F1-score de 0.17).

**Pivote Estratégico:**
En lugar de un problema binario, reformulamos el proyecto como un **problema multiclase** para predecir la gravedad del crimen, usando `law_cat_cd` como nuestro nuevo `TARGET`.

Analicemos la distribución de este nuevo target:

In [ ]:
# Analicemos la distribución de nuestro NUEVO target: law_cat_cd
if 'law_cat_cd' in df_eda.columns:
    plt.figure(figsize=(10, 6))
    law_counts = df_eda['law_cat_cd'].value_counts()
    sns.barplot(x=law_counts.index, y=law_counts.values, palette='colorblind', hue=law_counts.index, legend=False)
    plt.title('Distribución de Crímenes por Gravedad (Nuestro Target)', fontsize=16)
    plt.ylabel('Número de Incidentes')
    plt.xlabel('Categoría de Ley')
    plt.show()
else:
    print("La columna 'law_cat_cd' no se encontró (puede haber sido eliminada en una ejecución anterior).")

## 3. Análisis de Características (Features)

Ahora que tenemos nuestro objetivo (`law_cat_cd`), exploremos nuestros predictores (`features`) para entender los patrones.

### ¿Cuáles son los crímenes más comunes?

In [14]:
print("Análisis de los tipos de crímenes más comunes:")

# Contamos la frecuencia de cada tipo de crimen y seleccionamos el top 15
crime_counts = df_eda['ofns_desc'].value_counts().nlargest(15)

# Creamos la figura para el gráfico
plt.figure(figsize=(12, 8))

# --- CORRECCIÓN ESTÉTICA (Warning) ---
# Añadimos 'hue' y 'legend=False' para eliminar el FutureWarning
ax = sns.barplot(x=crime_counts.values, y=crime_counts.index, palette='viridis', hue=crime_counts.index, legend=False)

# Añadimos títulos y etiquetas para mayor claridad
ax.set_title('Top 15 Crímenes más Frecuentes en NYC (2020 en adelante)', fontsize=16)
ax.set_xlabel('Número de Incidentes Reportados', fontsize=12)
ax.set_ylabel('Tipo de Crimen', fontsize=12)

plt.show()

Análisis de los tipos de crímenes más comunes:


C:\Users\Diegosuar\AppData\Local\Temp\ipykernel_22136\4166725529.py:18: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


### ¿Cuándo ocurren los crímenes? (Análisis Temporal)

In [15]:
print("Análisis de patrones temporales de crímenes...")

# --- CORRECCIÓN LÓGICA ---
# Las columnas 'hour_of_day' y 'day_of_week' ahora se crean en la Celda 2,
# pero las crearemos de nuevo aquí por si acaso y para asegurar que el notebook funcione celda a celda.

df_eda['hour_of_day'] = df_eda['cmplnt_fr_dt'].dt.hour
df_eda['day_of_week'] = df_eda['cmplnt_fr_dt'].dt.day_name()

# Creamos una figura con dos subgráficos
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 12))

# --- CORRECCIÓN ESTÉTICA (Warning) ---
# Gráfico 1: Crímenes por hora del día
# Añadimos 'hue' y 'legend=False'
sns.countplot(x='hour_of_day', data=df_eda, ax=ax1, palette='plasma', hue='hour_of_day', legend=False)
ax1.set_title('Distribución de Crímenes por Hora del Día', fontsize=14)
ax1.set_xlabel('Hora (0-23)')
ax1.set_ylabel('Número de Incidentes')

# --- CORRECCIÓN ESTÉTICA (Warning) ---
# Gráfico 2: Crímenes por día de la semana
days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
# Añadimos 'hue', 'hue_order' y 'legend=False'
sns.countplot(x='day_of_week', data=df_eda, ax=ax2, order=days_order, palette='magma', hue='day_of_week', hue_order=days_order, legend=False)
ax2.set_title('Distribución de Crímenes por Día de la Semana', fontsize=14)
ax2.set_xlabel('Día de la Semana')
ax2.set_ylabel('Número de Incidentes')

# Ajustamos el espaciado y mostramos los gráficos
plt.tight_layout()
plt.show()

Análisis de patrones temporales de crímenes...


C:\Users\Diegosuar\AppData\Local\Temp\ipykernel_22136\48447853.py:32: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## 4. Análisis Geoespacial

Finalmente, ¿dónde ocurren los crímenes? Un mapa de calor nos mostrará las "zonas calientes" (hotspots) de actividad criminal en la ciudad.

In [8]:
# CELDA 5: Análisis Geoespacial (CORREGIDA)

import os # Importamos la librería para manejar el sistema de archivos

print("Generando mapa de calor de la densidad de crímenes...")
print("Este proceso puede tardar un poco si hay muchos datos.")

# Tomamos una muestra más pequeña para que el mapa cargue rápido
map_sample = df_eda.sample(n=20000, random_state=42)

# Coordenadas del centro de NYC
nyc_center_coords = [40.7128, -74.0060]

# Creamos el mapa base
nyc_heatmap = folium.Map(location=nyc_center_coords, zoom_start=11)

# Preparamos los datos para la capa de calor
heat_data = [[row['latitude'], row['longitude']] for index, row in map_sample.iterrows()]

# Añadimos la capa de calor al mapa
HeatMap(heat_data, radius=12).add_to(nyc_heatmap)

# --- LA SOLUCIÓN ESTÁ AQUÍ ---
# Nos aseguramos de que la carpeta 'reports' exista antes de guardar.
# exist_ok=True evita un error si la carpeta ya fue creada.
os.makedirs('reports', exist_ok=True) 

# Guardamos el mapa como un archivo HTML para el reporte
nyc_heatmap.save('reports/crime_density_heatmap.html')

print("\n¡Éxito! Mapa de calor guardado en 'reports/crime_density_heatmap.html'.")
# Mostramos el mapa en el notebook
nyc_heatmap

Generando mapa de calor de la densidad de crímenes...
Este proceso puede tardar un poco si hay muchos datos.

¡Éxito! Mapa de calor guardado en 'reports/crime_density_heatmap.html'.
